In [1]:
from pyspark import SparkContext, SparkConf

# conf = SparkConf().setAppName("Upload to Elasticsearch").setMaster("spark://25.19.134.189:7077")
# sc = SparkContext.getOrCreate(conf=conf)

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Upload Comment to HDFS').getOrCreate()

In [12]:
import pandas as pd
dataTest = pd.read_csv('../CrawlData/Data/res.csv')

In [65]:
data = spark.createDataFrame(dataTest.astype(str))

In [66]:
data.coalesce(1).write.mode('append').parquet('hdfs://cris:9000/test/')

In [4]:
url = 'hdfs://cris:9000/shopee/Product/part-00000-000e7568-4104-40bf-a899-dbe957403d8e-c000.snappy.parquet'
parDF2=spark.read.parquet(url)

In [5]:
parDF2.toPandas()

,itemid,shopid,userid,price_max_before_discount,has_lowest_price_guarantee,price_before_discount,price_min_before_discount,exclusive_price_info,hidden_price_display,price_min,...,is_non_cc_installment_payment_eligible,flash_sale,upcoming_flash_sale,deep_discount,has_low_fulfillment_rate,is_partial_fulfilled,makeups,shop_vouchers,global_sold,topic
0,3538808693,178567683,0,18500000000,False,18500000000,18500000000,None,None,11800000000,...,False,None,None,None,False,False,None,None,None,11036194
1,792675949,12004525,0,59000000000,False,59000000000,59000000000,None,None,57800000000,...,False,None,None,None,False,False,None,None,None,11036194
2,1327229205,65517936,0,19900000000,False,19900000000,19900000000,None,None,16900000000,...,False,None,None,None,False,False,None,None,None,11036194
3,748100791,25310465,0,48000000000,False,48000000000,48000000000,None,None,35900000000,...,False,None,None,None,False,False,None,None,None,11036194
4,3747177991,96135171,0,-1,False,0,-1,None,None,1600000000,...,False,None,None,None,False,False,None,None,None,11036194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4996,10539793087,266358040,0,800000000,False,600000000,600000000,None,None,330000000,...,False,None,None,None,False,False,None,None,None,11036345
4997,2329438255,149146795,0,-1,False,0,-1,None,None,840000000,...,False,None,None,None,False,False,None,None,None,11036345
4998,1674964833,30671159,0,5000000000,False,4000000000,4000000000,None,None,2600000000,...,False,None,None,None,False,False,None,None,None,11036345
4999,7123274356,27202429,0,5000000000,True,4500000000,4500000000,None,None,2480000000,...,False,None,None,None,False,False,None,None,None,11036345


In [7]:
data = parDF2.toJSON().collect()

In [1]:
from elasticsearch import Elasticsearch, helpers
import csv

In [2]:
es = Elasticsearch(host = "25.19.134.189", port = 9200,timeout=30,max_retries=10, retry_on_timeout=True)

In [69]:
i = 1
for item in data :

    es.index(index='testv4', doc_type='data', id=i, body=item)

    i = i + 1

<ipython-input-69-10ed49025917>:4: DeprecationWarning: The 'body' parameter is deprecated for the 'index' API and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.index(index='testv4', doc_type='data', id=i, body=item)
